In [1]:
using BenchmarkTools
versioninfo()

Julia Version 1.10.0
Commit 3120989f39 (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 12 × Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake)
  Threads: 18 on 12 virtual cores
Environment:
  JULIA_DEPOT_PATH = D:\.julia
  JULIA_NUM_PRECOMPILE_TASKS = 4
  JULIA_NUM_THREADS = 12
  JULIA_PYTHONCALL_EXE = D:\.julia\conda\3\python.exe


In [2]:
function f(L=10^8)
    s = 0.0
    for k in 1:L
        s += sin(k)/k
    end
    1 + 2s
end

@btime f()

  2.654 s (0 allocations: 0 bytes)


3.1415926695599605

In [3]:
@code_native debuginfo=:none f()

	.text
	.file	"f"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_f_1056
.LCPI0_0:
	.quad	0x3ff0000000000000              # double 1
	.text
	.globl	julia_f_1056
	.p2align	4, 0x90
	.type	julia_f_1056,@function
julia_f_1056:                           # @julia_f_1056
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	rsi
	push	rdi
	sub	rsp, 64
	vmovapd	xmmword ptr [rbp - 32], xmm7    # 16-byte Spill
	vmovapd	xmmword ptr [rbp - 48], xmm6    # 16-byte Spill
	.cfi_offset rdi, -32
	.cfi_offset rsi, -24
	.cfi_offset xmm6, -64
	.cfi_offset xmm7, -48
	vxorpd	xmm7, xmm7, xmm7
	mov	esi, 1
	movabs	rdi, offset j_sin_1058
	.p2align	4, 0x90
.LBB0_1:                                # %L2
                                        # =>This Inner Loop Header: Depth=1
	vcvtsi2sd	xmm6, xmm1, rsi
	vmovapd	xmm0, xmm6
	call	rdi
	vdivsd	xmm0,

In [4]:
using LoopVectorization

"single-thread optimization"
function f_turbo(L=10^8)
    s = 0.0
    @turbo for k in 1:L
        s += sin(k)/k
    end
    1 + 2s
end

@btime f_turbo()

  210.931 ms (0 allocations: 0 bytes)


3.1415926695577863

In [5]:
using LoopVectorization

"multi-thread optimization"
function f_tturbo(L=10^8)
    s = 0.0
    @tturbo for k in 1:L
        s += sin(k)/k
    end
    1 + 2s
end

@show Threads.nthreads()
@btime f_tturbo()

Threads.nthreads() = 12
  43.759 ms (0 allocations: 0 bytes)


3.1415926695578773

In [6]:
@code_native debuginfo=:none f_turbo(10^8)

	.text
	.file	"f_turbo"
	.section	.rodata.cst32,"aM",@progbits,32
	.p2align	5                               # -- Begin function julia_f_turbo_2868
.LCPI0_0:
	.quad	0x0000000000000000              # double 0
	.quad	0x3ff0000000000000              # double 1
	.quad	0x4000000000000000              # double 2
	.quad	0x4008000000000000              # double 3
.LCPI0_22:
	.quad	0                               # 0x0
	.quad	1                               # 0x1
	.quad	2                               # 0x2
	.quad	3                               # 0x3
.LCPI0_23:
	.quad	0                               # 0x0
	.quad	-1                              # 0xffffffffffffffff
	.quad	-2                              # 0xfffffffffffffffe
	.quad	-3                              # 0xfffffffffffffffd
.LCPI0_24:
	.long	0                               # 0x0
	.long	2                               # 0x2
	.long	4                               # 0x4
	.long	6                               # 0x6
	.zero	4
	.zero	4
	.zero	

In [7]:
@macroexpand function f_turbo(L=10^8)
    s = 0.0
    @turbo for k in 1:L
        s += sin(k)/k
    end
    1 + 2s
end

:(function f_turbo(L = 10 ^ 8)
      #= In[7]:1 =#
      #= In[7]:2 =#
      s = 0.0
      #= In[7]:3 =#
      begin
          begin
              #= In[7]:5 =#
              nothing
          end
          begin
              #= In[7]:4 =#
              nothing
          end
          begin
              #= In[7]:3 =#
              nothing
          end
          var"###range###1###" = static(1):L
          var"###length###2###" = StaticArrayInterface.static_length(var"###range###1###")
          if LoopVectorization.check_args(typeof(s)) && (true && ((LoopVectorization.can_turbo)(LoopVectorization.sin_fast, Val{1}()) && ((LoopVectorization.can_turbo)(LoopVectorization.div_fast, Val{2}()) && $(Expr(:&&, :((LoopVectorization.can_turbo)(LoopVectorization.add_fast, Val{2}())))))))
              var"####grouped#strided#pointer####5###" = (getfield)(LoopVectorization.grouped_strided_pointer((), Val{()}()), 1)
              begin
                  $(Expr(:gc_preserve, :(var"##s_##onevec##" 

In [8]:
@time f_tturbo(10^9)

  0.461977 seconds (2 allocations: 32 bytes)


3.141592652601785